In [1]:
%load_ext autoreload
%autoreload 2

<p align="center">
  <img src="imgs/QR.png" alt="QR" width="250"/>
</p>

# Einführung in die Lineare Regression: Von der Theorie zur Praxis


### Lernziele:

1.  **Verständnis der Linearen Regression**: Sie werden das Konzept der linearen Regression und das Ziel der Minimierung der Fehlerquadratsumme (OLS) verstehen.

2.  **Implementierung verschiedener Lösungsansätze**: Sie werden drei verschiedene Methoden zur Berechnung der Regressionsparameter kennenlernen und implementieren:
    *   **Analytische Lösung**: Direkte Berechnung der Parameter mittels statistischer Formeln.
    *   **Matrix-Form**: Lösung des Problems mit linearer Algebra.
    *   **Gradientenverfahren**: als Ihre Hauptaufgabe: die Implementierung des Gradientenverfahrens. Dies ist ein iterativer Optimierungsalgorithmus, der die Grundlage für viele komplexe Machine-Learning-Modelle bildet.


## Advertising-Datensatz: Variablen verstehen

Anhand des "Advertising"-Datensatzes analysieren wir den Zusammenhang zwischen Werbeausgaben und Verkaufszahlen. Der Datensatz enthält Informationen aus 200 verschiedenen Märkten.

### Variablen im Detail:

*   **Zielvariable (abhängige Variable):**
    *   `sales`: Die Verkäufe des Produkts in jedem Markt (in Tausend Einheiten). Dies ist die Variable, die wir vorhersagen möchten.

*   **Unabhängige Variablen (Features oder Prädiktoren):**
    *   `TV`: Das Werbebudget für das Fernsehen.
    *   `radio`: Das Werbebudget für das Radio.
    *   `newspaper`: Das Werbebudget für Zeitungen.

### Fokus:

Heute konzentrieren wir uns auf ein einfaches Modell, das nur den Zusammenhang zwischen TV-Werbung und Verkäufen betrachtet.
-> Wir wollen herausfinden, wie die TV-Werbeausgaben die Verkaufszahlen (`sales`) beeinflussen. 

Dazu verwenden wir ein einfaches mathematisches Modell:

$$
\text{sales} = \beta_0 + \beta_1 \times \text{TV}
$$

In dieser Gleichung sind:

*   **β₀** der **Achsenabschnitt (Intercept)**: Der geschätzte Verkaufswert, wenn keine TV-Werbung geschaltet wird.
*   **β₁** die **Steigung (Slope)**: zeigt, wie stark die Verkäufe steigen, wenn das TV-Budget um eine Einheit größer wird.

Diese beiden Werte – β₀ und β₁ – nennt man Modellparameter. Wir wollen jetzt herausfinden, wie man die optimalen Werte für diese Parameter anhand der Daten schätzen kann.

In [2]:
# Advertising-Datensatz einlesen
import pandas as pd

df = pd.read_csv('data/Advertising.csv', index_col=0)
df.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [3]:
# TV und Sales Spalten für weitere Analyse extrahieren
df_tv_sales = df[['TV', 'sales']]  

# Die ersten Zeilen anzeigen
df_tv_sales.head()  

,TV,sales
1,230.1,22.1
2,44.5,10.4
3,17.2,9.3
4,151.5,18.5
5,180.8,12.9


In [4]:
# Scatterplot für TV vs Sales erstellen
import plotly.express as px
fig = px.scatter(df_tv_sales, x='TV', y='sales', title='TV Advertising vs Sales',
                 labels={'TV': 'TV Advertising ($)', 'sales': 'Sales'})
fig.update_layout(
    width=600,
    height=500,
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18)
)
fig.show()

Unsere Aufgabe ist es, die **beste Gerade** zu finden, die die Verkäufe anhand des TV-Werbebudgets vorhersagt.

$$
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x
$$

Das bedeutet: Wir suchen einen **Achsenabschnitt** $\hat{\beta}_0$ und eine **Steigung** $\hat{\beta}_1$, sodass die Linie möglichst nah an den n = 200 Datenpunkten liegt.

👉 Dafür nutzen wir das Kleinste-Quadrate-Kriterium (Ordinary Least Squares, OLS)

---

### Ordinary Least Squares (OLS):

In [5]:
# OLS - Sowohl allgemein als auch TV Sales in einem Tool!
from utils.least_squares_visualizer import run_combined_visualizer


combined_viz = run_combined_visualizer(df_tv_sales)

## Fehlerquadratsumme (SSE) minimieren

Wir wollen die **Summe der quadrierten Fehler (SSE)** so klein wie möglich machen:

$$
SSE = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 
     = \sum_{i=1}^{n} (y_i - \beta_0 - \beta_1 x_i)^2
$$

**Wie bestimmen wir β₀ und β₁, die SSE minimieren?**

### **OLS: Statistischer Ansatz (Ableitungsbasiert)**

1. **Idee:**

   * SSE nach β₀ und β₁ **ableiten**,
   * Gleichungen = 0 setzen → **Normalgleichungen**.

2. **Lösung:**

   * **Steigung (β₁):**

     $$
     \hat{\beta}_1 = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sum (x_i - \bar{x})^2} 
     = \frac{\text{Cov}(X,Y)}{\text{Var}(X)}
     $$
   * **Achsenabschnitt (β₀):**

     $$
     \hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}
     $$

3. **Finales Modell:**

   $$
   \hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x
   $$

👉 $\hat{\beta}_0, \hat{\beta}_1$ sind **Schätzwerte** für die unbekannten wahren Parameter.




In [ ]:
# Analytische Lösung für OLS-Regression
import numpy as np
def ols_analytical(X, y):
    """
    Berechnet den Achsenabschnitt und die Steigung einer einfachen linearen Regression 
    mit der analytischen Lösung.
    Parameter:
    X : numpy array oder pandas Series
        Die unabhängige Variable (Prädiktor).
    y : numpy array oder pandas Series
        Die abhängige Variable (Zielvariable).
    Rückgabe:
    intercept : float
        Der Achsenabschnitt der Regressionsgerade.
    slope : float
        Die Steigung der Regressionsgerade.
    """
    # YOUR CODE HERE            
    
    return intercept, slope

In [ ]:
import numpy as np
X = df_tv_sales['TV']
y = df_tv_sales['sales']


beta_0, beta_1 = ols_analytical(X, y)

print(f"Intercept: {beta_0}, Slope: {beta_1}")

# Vorhergesagte Sales-Spalte erstellen
y_hat = beta_0 + beta_1 * df_tv_sales['TV']

In [8]:
# Scatterplot von TV vs Sales und die angepasste Linie mit plotly erstellen
import plotly.express as px
fig = px.scatter(df_tv_sales, x='TV', y='sales', title='TV Advertising vs Sales',
                 labels={'TV': 'TV Advertising ($)', 'sales': 'Sales'})
fig.add_scatter(x=X, y=y_hat, mode='lines', name='Fitted Line', line=dict(color='red'))
fig.update_layout(
    width=600,
    height=500,
    xaxis_title_font=dict(size=18),
    yaxis_title_font=dict(size=18)
)
fig.show()

Die vorherige Abbildung zeigt die Anpassung der einfachen linearen Regression an die *Advertising*-Daten, wobei $\hat{\beta}_0 = 7.03$ und $\hat{\beta}_1 = 0.0475$ sind. Mit anderen Worten: Nach diesem Modell ist eine zusätzliche Ausgabe von **1.000 $** für TV-Werbung mit einer **Erhöhung des Verkaufs um etwa 47,5 Tausend Dollar** verbunden.


### 2. OLS – Lineare Algebra Ansatz (Matrix-Form)

Mit **Matrixschreibweise** können wir das Regressionsproblem allgemein darstellen.
Das macht es leichter, später auf **multiple Regression** zu erweitern.

Ziel: Wir wollen die **besten Schätzungen** finden für

$$
\hat{\beta}_0 \quad \text{und} \quad \hat{\beta}_1
$$

(das sind Achsenabschnitt und Steigung).


#### Matrix-Formulierung

$$
\mathbf{y} = \mathbf{X} \hat{\boldsymbol{\beta}} 
$$

* **y** = Zielvektor $(n \times 1)$
* **X** = Designmatrix $(n \times 2)$, enthält eine Spalte mit Einsen für den Achsenabschnitt
* **β** = Parametervektor $(2 \times 1)$


$$
\hat{\boldsymbol{\beta}} = 
\begin{bmatrix} 
\hat{\beta}_0 \\ 
\hat{\beta}_1 
\end{bmatrix}, \quad 
\mathbf{X} = 
\begin{bmatrix} 
1 & x_1 \\ 
1 & x_2 \\ 
\vdots & \vdots \\ 
1 & x_n 
\end{bmatrix}, \quad 
\mathbf{y} = 
\begin{bmatrix} 
y_1 \\ 
y_2 \\ 
\vdots \\ 
y_n 
\end{bmatrix}
$$

#### Normalengleichung

$$
\hat{\boldsymbol{\beta}} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

In [ ]:
def ols_matrix_form(X, y):
    """    
    Berechnet den Achsenabschnitt und die Steigung einer einfachen linearen Regression 
    mit der Matrixform der Normalengleichung.
    Parameter:
    X : numpy array oder pandas Series
        Die unabhängige Variable (Prädiktor).
    y : numpy array oder pandas Series
        Die abhängige Variable (Zielvariable).
    Rückgabe:
    intercept : float
        Der Achsenabschnitt der Regressionsgerade.
    slope : float
        Die Steigung der Regressionsgerade.
    """
    # In Matrixformat konvertieren: y = Xβ + ε
    # Wobei β = [intercept, slope]
    X_matrix = np.column_stack([np.ones(len(X)), X])  # Spalte mit Einsen hinzufügen
                                                      # Erstellt 2D-Matrix

    
    # Normalengleichung: β = (X'X)⁻¹X'y
    # YOUR CODE HERE
    
    return intercept, slope

In [ ]:
X = df_tv_sales['TV']
y = df_tv_sales['sales']


beta_0, beta_1 = ols_matrix_form(X, y)

print(f"Intercept: {beta_0}, Slope: {beta_1}")

# Vorhergesagte Sales-Spalte erstellen
y_hat = beta_0 + beta_1 * df_tv_sales['TV']

👉 Vorteil vom Matrix-Ansatz:
Er findet die optimale Lösung immer exakt.
Nachteil: Bei sehr großen Datensätzen ist das Rechnen mit der Inversen sehr teuer.
Und manchmal kann die Matrix gar nicht invertiert werden.
Darum nutzt man in solchen Fällen **iterative Methoden**.

### 3. OLS: Iterative Methoden (Gradient Descent)

Im Gegensatz zu **analytischen Lösungen** wie der Normalengleichung, die das Ergebnis in einem Schritt berechnen, sind **Gradient Descent Methoden** **iterative Verfahren**.
Das heißt: Die Parameter werden Schritt für Schritt verbessert, bis der **Fehler möglichst klein** ist.


#### Mathematische Grundlage

**Ziel:** Kostenfunktion minimieren

$$
J(\hat{\beta}_0, \hat{\beta}_1) = \frac{1}{n} \sum_{i=1}^{n} (y_i - (\hat{\beta}_0 + \hat{\beta}_1 x_i))^2
$$


#### **Gradientenabstiegs-Algorithmus**

1. **Parameter initialisieren** $\hat{\beta}_0, \hat{\beta}_1$ (z. B. Nullen oder Zufallswerte).  
2. **beginnt eine Schleife:**

   1. **Gradienten berechnen** der Kostenfunktion bezüglich jedes Parameters:  
   > Wie stark ändert sich der Fehler, wenn wir einen der Parameter verändern? Das nennt man den **Gradienten** – also die Steigung der Kostenfunktion.
   > Für den Achsenabschnitt $\hat{\beta}_0$ und für die Steigung $\hat{\beta}_1$ gibt es jeweils eine Formel:

      Für $\hat{\beta}_0$ (Achsenabschnitt):  
      $$
      \frac{\partial J}{\partial \hat{\beta}_0} = \frac{2}{n} \sum_{i=1}^{n} -(y_i - (\hat{\beta}_0 + \hat{\beta}_1 x_i))
      $$

      Für $\hat{\beta}_1$ (Steigung):  
      $$
      \frac{\partial J}{\partial \hat{\beta}_1} = \frac{2}{n} \sum_{i=1}^{n} -x_i(y_i - (\hat{\beta}_0 + \hat{\beta}_1 x_i))
      $$

   2. **Parameter aktualisieren** (Update-Regel):  
   > Wir passen die Parameter an – also wir gehen einen kleinen Schritt in die Richtung, in der der Fehler kleiner wird.  
      $$
      \hat{\beta}_0^{(new)} = \hat{\beta}_0^{(old)} - \alpha \frac{\partial J}{\partial \hat{\beta}_0}
      $$

      $$
      \hat{\beta}_1^{(new)} = \hat{\beta}_1^{(old)} - \alpha \frac{\partial J}{\partial \hat{\beta}_1}
      $$ 

      Der Gradient zeigt uns **wie stark und in welche Richtung** sich die Kosten ändern, wenn sich der Parameter verändert:  

      * Ist der Gradient **positiv**, verringere $\hat{\beta}_j$ (nach links gehen).  
      * Ist der Gradient **negativ**, erhöhe $\hat{\beta}_j$ (nach rechts gehen).  

   4. **Neuen Fehlerwert berechnen** $J(\hat{\beta}_0, \hat{\beta}_1)$.  
   5. **Abbruchbedingung prüfen** (Kostenänderung < Toleranz oder maximale Iterationen erreicht).  


#### **Aufgabe: Implementierung**

Implementieren Sie Gradientenverfahren für lineare Regression.

In [11]:
def ols_gradient_descent(X, y, learning_rate=0.0001, max_iterations=10000):
    """
    Implementierung des Gradientenabstiegs für lineare Regression mit MSE-Kostenfunktion.
    
    Parameter:
    X : array-like, shape (n_samples,)
        Die unabhängige Variable (Prädiktor)
    y : array-like, shape (n_samples,)
        Die abhängige Variable (Zielwert)
    learning_rate : float, default=0.0001
        Schrittgröße für den Gradientenabstieg
    max_iterations : int, default=10000
        Maximale Anzahl der Iterationen
    
    Rückgabe:
    intercept : float
        Der Achsenabschnitt der Regressionsgerade (β₀)
    slope : float
        Die Steigung der Regressionsgerade (β₁)
    
    Tipps:
    1. Initialisieren Sie beide Parameter (Steigung und Achsenabschnitt) mit 0
    2. Für jede Iteration:
       - Berechnen Sie Vorhersagen mit aktuellen Parametern: ŷ = β₀ + β₁ * X
       - Berechnen Sie Gradienten für beide Parameter:
         * Für Steigung (β₁): ∂J/∂β₁ = (-2/n) * Σ[X_i * (y_i - ŷ_i)]
         * Für Achsenabschnitt (β₀): ∂J/∂β₀ = (-2/n) * Σ[(y_i - ŷ_i)]
       - Aktualisieren Sie Parameter: β_neu = β_alt - Lernrate * Gradient
    3. Die Anzahl der Stichproben n = len(X)
    4. Verwenden Sie np.sum() für die Gradientenberechnung
    """
    
    # TODO: Parameter initialisieren
    beta_0 = 0  # Achsenabschnitt
    beta_1 = 0  # Steigung

    # TODO: Anzahl der Stichproben ermitteln
    
    
    # TODO: Gradientenabstieg-Schleife implementieren
    # Tipp: Verwenden Sie eine for-Schleife mit max_iterations
    for i in range(max_iterations):
        # TODO: Vorhersagen berechnen
        
        
        # TODO: Gradient für Steigung (β₁) berechnen
        
        
        # TODO: Gradient für Achsenabschnitt (β₀) berechnen
        
        
        # TODO: Parameter mit Gradientenabstieg-Regel aktualisieren
        
        
        pass
    
    # TODO: Die finalen Parameter zurückgeben
    # Tipp: return achsenabschnitt, steigung
    pass



In [13]:
# Testen Sie Ihre Implementierung
X = df_tv_sales['TV']
y = df_tv_sales['sales']
beta_0, beta_1 = ols_gradient_descent(X, y)
print(f"Achsenabschnitt: {beta_0:.4f}, Steigung: {beta_1:.4f}")

c:\Users\afi89\Documents\Students\Machine Learning\Regression_Tasks\.venv\lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning:

overflow encountered in reduce

C:\Users\afi89\AppData\Local\Temp\ipykernel_26012\183192252.py:57: RuntimeWarning:

invalid value encountered in scalar subtract



Achsenabschnitt: inf, Steigung: nan


Haben Sie das folgende Output?:
```python
invalid value encountered in scalar subtract

Achsenabschnitt: inf, Steigung: nan
```

Die Implementierung war nicht optimal, da es zu numerischen Überläufen kommt (die Gradienten werden zu groß und verursachen ein "Explodieren" der Parameter). Daher verwenden wir Feature-Skalierung:

### Warum passiert das?

Das Problem liegt in den **unterschiedlichen Größenordnungen** der Daten:
- **TV-Werbebudget**: Werte zwischen 0 und 300 (Größenordnung: Hunderte)
- **Verkäufe**: Werte zwischen 1 und 27 (Größenordnung: Zehner)

Ohne Skalierung werden die Gradienten sehr groß, besonders für β₁ (TV-Koeffizient), da: 
- ∂J/∂β₁:`d_beta_1 = (-2/n) * np.sum(X * (y - y_pred))`
- Hier wird X (TV-Werte bis 300) mit den Residuen multipliziert

### Die Lösung: Feature-Skalierung

**Feature-Skalierung** transformiert die Variablen so, dass sie ähnliche Wertebereiche haben. Dies:
- **Verhindert numerische Instabilität**
- **Beschleunigt die Konvergenz** des Gradientenabstiegs
- **Ermöglicht größere Lernraten** ohne Divergenz

**Wichtiger Hinweis**: Nach der Skalierung müssen die Parameter zurück auf die ursprüngliche Skala transformiert werden, damit die Interpretation stimmt.

Die folgende sichere Implementierung verwendet **Standardisierung** (Z-Score-Normalisierung):

In [14]:
def ols_gradient_descent_safe(X, y, learning_rate=0.01, max_iterations=10000):
    """
    Sichere Implementierung des Gradientenabstiegs für lineare Regression mit Feature-Skalierung.
    
    Parameter:
    X : array-like, shape (n_samples,)
        Die unabhängige Variable (Prädiktor)
    y : array-like, shape (n_samples,)
        Die abhängige Variable (Zielwert)
    learning_rate : float, default=0.01
        Schrittgröße für den Gradientenabstieg
    max_iterations : int, default=10000
        Maximale Anzahl der Iterationen
    
    Rückgabe:
    intercept : float
        Der Achsenabschnitt der Regressionsgerade (β₀)
    slope : float
        Die Steigung der Regressionsgerade (β₁)
    """
    # Feature-Skalierung - X standardisieren
    X_mean = np.mean(X)
    X_std = np.std(X)
    X_scaled = (X - X_mean) / X_std # Z-Score-Normalisierung
    
    # Y ebenfalls skalieren für bessere numerische Stabilität
    y_mean = np.mean(y)
    y_scaled = y - y_mean           # Nur zentrieren, keine Division
    
    # Parameter initialisieren
    beta_1 = 0  # Steigung
    beta_0 = 0  # Achsenabschnitt
    n = len(X_scaled)  # Anzahl der Stichproben
    
    # Gradientenabstieg mit skalierten Features
    for i in range(max_iterations):
        # Vorhersagen mit aktuellen Parametern berechnen
        y_pred = beta_0 + beta_1 * X_scaled
        
        # Gradienten für beide Parameter berechnen
        d_beta_1 = (-2/n) * np.sum(X_scaled * (y_scaled - y_pred))
        d_beta_0 = (-2/n) * np.sum(y_scaled - y_pred)
        
        # Parameter mit Gradientenabstieg-Regel aktualisieren
        beta_1 = beta_1 - learning_rate * d_beta_1
        beta_0 = beta_0 - learning_rate * d_beta_0
    
    # Parameter zurück auf die ursprüngliche Skala transformieren
    beta_1_original = beta_1 / X_std
    beta_0_original = (beta_0 + y_mean) - beta_1_original * X_mean
    
    return beta_0_original, beta_1_original

In [15]:
# Testen Sie Ihre Implementierung
X = df_tv_sales['TV']
y = df_tv_sales['sales']
beta_0, beta_1 = ols_gradient_descent_safe(X, y)
print(f"Achsenabschnitt: {beta_0:.4f}, Steigung: {beta_1:.4f}")

Achsenabschnitt: 7.0326, Steigung: 0.0475
